In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sb
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split 

import numpy as np
import pandas as pd 
import math
import matplotlib.pyplot as plt
import seaborn as sbn
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import Ridge, Lasso
from sklearn.grid_search import GridSearchCV
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
plt.style.use('seaborn-white')

## Dataframe construction

In [3]:
import pandas as pd

In [4]:
import pandas as pd
df1 = pd.read_csv('../Part 2 Capstone/df2.csv', index_col=0)

In [5]:
df1['MONTH_NAME'] = df1['MONTH_NAME'].apply(lambda x: x.replace(' ', ''))

In [6]:
event_types = ['Drought' ,'Flood' ,'Winter Storm', 'OTHER', 'High Wind' ,'ThunderStorm',
 'Wildfire', 'Rip Current' ,'Tropical Storm' ,'Tornado' ,'Hurricane', 'Tsunami',
 'Landslide']
for i in event_types:
     df1['is_'+i] = df1['EVENT_TYPE'].apply(lambda x: 1 if i in x else 0)

In [7]:
df1.TOR_F_SCALE.unique()

array([nan, 'F1', 'F0', 'F2', 'F3', 'F4', 'EF3', 'EF1', 'EF0', 'EF2',
       'EF4', 'EF5'], dtype=object)

In [8]:
tor_all = pd.get_dummies(df1.TOR_F_SCALE, prefix="is", prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)
tor_all_1 = pd.get_dummies(df1.STATE, prefix="is", prefix_sep='_', dummy_na=False, columns=None, sparse=False, drop_first=False)

In [9]:
# tor_all.head()
tor_all_1.head()

,is_ALABAMA,is_ALASKA,is_AMERICAN SAMOA,is_ARIZONA,is_ARKANSAS,is_ATLANTIC NORTH,is_ATLANTIC SOUTH,is_CALIFORNIA,is_COLORADO,is_CONNECTICUT,...,is_TENNESSEE,is_TEXAS,is_UTAH,is_VERMONT,is_VIRGIN ISLANDS,is_VIRGINIA,is_WASHINGTON,is_WEST VIRGINIA,is_WISCONSIN,is_WYOMING
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df1['TD'] = df1['TOTAL_DEATHS'].apply(lambda x: 1 if x > 0 else 0)

In [11]:
df1['TI'] = df1['TOTAL_INJURIES'].apply(lambda x: 1 if x > 0 else 0)

In [12]:
df1['TI'].describe()

count    654313.000000
mean          0.010668
std           0.102732
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: TI, dtype: float64

In [13]:
# is_tornado_model['CASUALTIES'].value_counts()

In [14]:
df_all = pd.concat([df1, tor_all, tor_all_1], axis=1)

In [15]:
df_all.describe()

,EVENT_ID,EPISODE_ID,INJURIES_DIRECT,INJURIES_INDIRECT,TOTAL_INJURIES,DEATHS_DIRECT,DEATHS_INDIRECT,TOTAL_DEATHS,CASUALTIES,DAMAGE_PROPERTY,...,is_TENNESSEE,is_TEXAS,is_UTAH,is_VERMONT,is_VIRGIN ISLANDS,is_VIRGINIA,is_WASHINGTON,is_WEST VIRGINIA,is_WISCONSIN,is_WYOMING
count,6.543130e+05,6.543130e+05,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,6.543130e+05,...,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000
mean,1.892418e+06,8.729241e+04,0.053577,0.013384,0.066960,0.008861,0.002017,0.010879,0.077839,3.812313e+05,...,0.023927,0.069227,0.006355,0.006674,0.000271,0.028853,0.005072,0.011657,0.023666,0.010217
std,2.418225e+06,7.913208e+04,2.914238,3.024913,4.203051,0.284431,0.062737,0.297602,4.334101,3.506985e+07,...,0.152823,0.253840,0.079463,0.081423,0.016445,0.167394,0.071041,0.107334,0.152007,0.100561
min,3.419000e+03,7.650000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.808540e+05,3.058300e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.533630e+05,5.888900e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.371730e+06,1.625420e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.535308e+06,1.152422e+06,1150.000000,2400.000000,2400.000000,158.000000,11.000000,161.000000,2409.000000,1.790000e+10,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [16]:
is_all_model = df_all

In [17]:
is_all_model.describe()

,EVENT_ID,EPISODE_ID,INJURIES_DIRECT,INJURIES_INDIRECT,TOTAL_INJURIES,DEATHS_DIRECT,DEATHS_INDIRECT,TOTAL_DEATHS,CASUALTIES,DAMAGE_PROPERTY,...,is_TENNESSEE,is_TEXAS,is_UTAH,is_VERMONT,is_VIRGIN ISLANDS,is_VIRGINIA,is_WASHINGTON,is_WEST VIRGINIA,is_WISCONSIN,is_WYOMING
count,6.543130e+05,6.543130e+05,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,6.543130e+05,...,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000,654313.000000
mean,1.892418e+06,8.729241e+04,0.053577,0.013384,0.066960,0.008861,0.002017,0.010879,0.077839,3.812313e+05,...,0.023927,0.069227,0.006355,0.006674,0.000271,0.028853,0.005072,0.011657,0.023666,0.010217
std,2.418225e+06,7.913208e+04,2.914238,3.024913,4.203051,0.284431,0.062737,0.297602,4.334101,3.506985e+07,...,0.152823,0.253840,0.079463,0.081423,0.016445,0.167394,0.071041,0.107334,0.152007,0.100561
min,3.419000e+03,7.650000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.808540e+05,3.058300e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.533630e+05,5.888900e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,5.371730e+06,1.625420e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,5.535308e+06,1.152422e+06,1150.000000,2400.000000,2400.000000,158.000000,11.000000,161.000000,2409.000000,1.790000e+10,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [18]:
is_all_model.drop([ 'YEAR', u'TOTAL_INJURIES', u'DAMAGE_PROPERTY', u'DAMAGE_CROPS',
       u'COMBINED_DAMAGE', 'EVENT_ID', u'EPISODE_ID',u'INJURIES_DIRECT', u'CASUALTIES', u'INJURIES_INDIRECT','EVENT_TYPE', 'DATE', u'CZ_NAME', u'CATEGORY', u'STATE',
       u'MONTH_NAME', u'TOR_F_SCALE', u'EPISODE_NARRATIVE', u'EVENT_NARRATIVE',u'DEATHS_DIRECT',
       u'DEATHS_INDIRECT', u'TOTAL_DEATHS'], axis=1, inplace=True)

In [19]:
is_all_model.columns

Index([u'TOR_LENGTH', u'TOR_WIDTH', u'is_Drought', u'is_Flood',
       u'is_Winter Storm', u'is_OTHER', u'is_High Wind', u'is_ThunderStorm',
       u'is_Wildfire', u'is_Rip Current', u'is_Tropical Storm', u'is_Tornado',
       u'is_Hurricane', u'is_Tsunami', u'is_Landslide', u'TD', u'TI',
       u'is_EF0', u'is_EF1', u'is_EF2', u'is_EF3', u'is_EF4', u'is_EF5',
       u'is_F0', u'is_F1', u'is_F2', u'is_F3', u'is_F4', u'is_ALABAMA',
       u'is_ALASKA', u'is_AMERICAN SAMOA', u'is_ARIZONA', u'is_ARKANSAS',
       u'is_ATLANTIC NORTH', u'is_ATLANTIC SOUTH', u'is_CALIFORNIA',
       u'is_COLORADO', u'is_CONNECTICUT', u'is_DELAWARE',
       u'is_DISTRICT OF COLUMBIA', u'is_E PACIFIC', u'is_FLORIDA',
       u'is_GEORGIA', u'is_GUAM', u'is_GULF OF ALASKA', u'is_GULF OF MEXICO',
       u'is_HAWAII', u'is_HAWAII WATERS', u'is_IDAHO', u'is_ILLINOIS',
       u'is_INDIANA', u'is_IOWA', u'is_KANSAS', u'is_KENTUCKY',
       u'is_LAKE ERIE', u'is_LAKE HURON', u'is_LAKE MICHIGAN',
       u'is_LAKE ONTA

In [38]:
is_all_model.TD.sum()

4481

In [39]:
rf_model = RandomForestClassifier()

y1 = LabelEncoder().fit_transform(is_all_model['TD'])
x1 = is_all_model.drop(['TI','TD'], axis=1)

model1 = rf_model.fit(x1,y1)
# model1.score(x1,y1)

In [21]:
predicted_class1 = model1.predict(x1)
confusion_matrix(y1, predicted_class1)

array([[649693,    139],
       [  3781,    700]])

In [22]:
print classification_report(y1, predicted_class1)

             precision    recall  f1-score   support

          0       0.99      1.00      1.00    649832
          1       0.83      0.16      0.26      4481

avg / total       0.99      0.99      0.99    654313



## Precision

    Precision is the number of True Positives divided by the number of True Positives and False Positives. Put another way, it is the number of positive predictions divided by the total number of positive class values predicted. It is also called the Positive Predictive Value (PPV).

    Precision can be thought of as a measure of a classifiers exactness. A low precision can also indicate a large number of False Positives.



## Recall

    Recall is the number of True Positives divided by the number of True Positives and the number of False Negatives. Put another way it is the number of positive predictions divided by the number of positive class values in the test data. It is also called Sensitivity or the True Positive Rate.

    Recall can be thought of as a measure of a classifiers completeness. A low recall indicates many False Negatives.


    
    F1 Score

    The F1 Score is the 2*((precision*recall)/(precision+recall)). It is also called the F Score or the F Measure. Put another way, the F1 score conveys the balance between the precision and the recall.


.

In [24]:
# plt.figure(figsize=(11,8))
# sbn.heatmap(is_tornado_model.corr())

In [23]:
x_train, x_test, y_train, y_test = train_test_split(x1,y1)

In [24]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((490734, 94), (163579, 94), (490734,), (163579,))

In [25]:
rfTTS = RandomForestClassifier()

In [26]:
modelTTS = rfTTS.fit(x_train, y_train)

modelTTS.score(x_train, y_train)

0.99405788064409639

In [27]:
predict_train = modelTTS.predict(x_train)
confusion_matrix(y_train, predict_train)

array([[487277,     99],
       [  2817,    541]])

In [28]:
modelTTS.score(x_test, y_test)

0.99347715782588231

In [30]:
predict_test = modelTTS.predict(x_test)
confusion_matrix(y_test, predict_test)


array([[162398,     58],
       [  1009,    114]])

In [32]:
# Y_score = rfTTS.decision_function(x_test)

# FPR = dict()
# TPR = dict()
# ROC_AUC = dict()

# print roc_curve(y_test, Y_score)

# FPR[1], TPR[1], _ = roc_curve(y_test, Y_score)
# ROC_AUC[1] = auc(FPR[1], TPR[1])

In [33]:
# plt.figure(figsize=[11,9])
# plt.plot(FPR[1], TPR[1], label='ROC curve (area = %0.2f)' % ROC_AUC[1], linewidth=4)
# plt.plot([0, 1], [0, 1], 'k--', linewidth=4)
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate', fontsize=18)
# plt.ylabel('True Positive Rate', fontsize=18)
# plt.title('Tornado Deaths Predictor', fontsize=18)
# plt.legend(loc="lower right")
# plt.show()

In [34]:
# #GRidsearch
# rf_parameters = {
#     "n_estimators" : [10, 25, 50, 100, 500],
#     "max_depth" : [2,3,5,7,10, None],
#     "max_features" : [0.25, 0.5, 0.75, 1.0]
# }
# modelgs = RandomForestClassifier()

In [35]:
# grid_search = GridSearchCV(estimator=modelgs,
#                           param_grid=rf_parameters
#                            , n_jobs=-1) 
# grid_search

In [36]:
# ygs = LabelEncoder().fit_transform(y_test)
# xgs = x_test

In [37]:
# grid_search.fit(xgs, ygs)

In [38]:
# grid_search.best_score_

In [39]:
# grid_search.best_estimator_

In [40]:
# grid_search.best_estimator_.fit(x1, y1)

In [41]:
# grid_search.best_estimator_.score(x1, y1)

In [42]:
# Confusion matrix for final grid-search model

# gspredict = grid_search.predict(xgs)
# confusion_matrix(ygs, gspredict)

In [31]:
df4 = is_all_model

In [32]:
#convert dataframe to csv

df4.to_csv('df4.csv')
#This worked!  Thank God this was successful on Tuesday school!
# 10 years worth of data (654313, 24)

In [43]:
df = is_all_model.sample(frac=0.1)
df.reset_index(drop=True, inplace=True)

In [44]:
df.describe()

,TOR_LENGTH,TOR_WIDTH,is_Drought,is_Flood,is_Winter Storm,is_OTHER,is_High Wind,is_ThunderStorm,is_Wildfire,is_Rip Current,...,is_TENNESSEE,is_TEXAS,is_UTAH,is_VERMONT,is_VIRGIN ISLANDS,is_VIRGINIA,is_WASHINGTON,is_WEST VIRGINIA,is_WISCONSIN,is_WYOMING
count,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,...,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000,65431.000000
mean,0.069032,3.762667,0.068286,0.104431,0.412526,0.010163,0.043756,0.311045,0.006067,0.001009,...,0.023674,0.069982,0.005991,0.006358,0.000352,0.029344,0.005319,0.012379,0.023659,0.010729
std,0.875917,49.205042,0.252237,0.305821,0.492293,0.100301,0.204553,0.462925,0.077658,0.031744,...,0.152032,0.255119,0.077170,0.079483,0.018746,0.168770,0.072735,0.110573,0.151984,0.103024
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,100.000000,2815.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [46]:
y1 = LabelEncoder().fit_transform(df['TD'])
x1 = df.drop(['TD'], axis=1)

In [47]:
# setting up train and test data for X and y
X_train, X_test, y_train, y_test = train_test_split(x1, y1, test_size=0.3)
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

(45801, 84) (45801,)
(19630, 84) (19630,)


In [51]:
global_models = {
   'knn' : KNeighborsClassifier(),
   'knn_bag' : BaggingClassifier(KNeighborsClassifier()),
   'logreg' : LogisticRegression(),
   'logreg_bag' : BaggingClassifier(LogisticRegression()),
   'tree' : DecisionTreeClassifier(),
   'tree_bag' : BaggingClassifier(DecisionTreeClassifier()),
   'forest' : RandomForestClassifier(),
   'extra_tree' : ExtraTreesClassifier(),
}

In [49]:
results = {}

def evaluate_model(X_train, X_test, 
                   y_train, y_test,
                  model):
    pick_a_model = global_models[model]
    pick_a_model.fit(X_train, y_train)
    print 'This is the model:', (model)
    print 'Train model score:', pick_a_model.score(X_train, y_train)
    predictions = pick_a_model.predict(X_test)
    print '\nConfusion matrix: \n\n', confusion_matrix(y_test, predictions)
    print '\nClassification report: \n\n', classification_report(y_test, predictions)
    results.update({model: pick_a_model.score(X_test, y_test)})


In [50]:
for key in global_models:
    evaluate_model(X_train, X_test, y_train, y_test, key)

This is the model: knn
Train model score: 0.993995764285

Confusion matrix: 

[[19503     2]
 [  111    14]]

Classification report: 

             precision    recall  f1-score   support

          0       0.99      1.00      1.00     19505
          1       0.88      0.11      0.20       125

avg / total       0.99      0.99      0.99     19630

This is the model: logreg_bag
Train model score: 0.993362590336

Confusion matrix: 

[[19500     5]
 [  117     8]]

Classification report: 

             precision    recall  f1-score   support

          0       0.99      1.00      1.00     19505
          1       0.62      0.06      0.12       125

avg / total       0.99      0.99      0.99     19630

This is the model: knn_bag
Train model score: 0.994017597869

Confusion matrix: 

[[19503     2]
 [  111    14]]

Classification report: 

             precision    recall  f1-score   support

          0       0.99      1.00      1.00     19505
          1       0.88      0.11      0.20     

In [1]:
#convert dataframe to csv

df.to_csv('df4.csv')
#This worked!  Thank God this was successful on Tuesday school!
# 10 years worth of data (654313, 24)

NameError: name 'df' is not defined

In [ ]:
## plots ROC curve
def plot_roc(x_test, y_test, model):
    fpr, tpr, _ = roc_curve(y_test, model.predict_proba(x_test)[:,1])
    roc_auc = auc(fpr, tpr)
    print 'ROC AUC: %0.2f' % roc_auc
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    


In [ ]:
for model in global_models:
    plot_roc(X_test, y_test, model)